In [1]:
from bs4 import BeautifulSoup
import re
import requests
from sqlmodel import SQLModel, Field, create_engine, Session
from typing import Optional
import json

### Extraction and preprocessing functions
Firstly, I'll extract all columns from the [main page](https://en.wikipedia.org/wiki/List_of_highest-grossing_films) table using extract_highest_grossing_films.
Secondly, for each extracted row, I'll get additional info from film's page in wiki_page_link via extract_film_details.
Lastly, I'll combine data from two previous steps in Film object.


In [2]:
citations_pattern = re.compile("\[[0-9]+\]")
def preprocess_text(text):
    """
    Remove citations and newlines from text.
    """
    text = text.strip().replace('\n', ' ')
    text = citations_pattern.sub('', text)
    return text.strip()

In [3]:
def extract_highest_grossing_films(html_content):
    """
    Extract data from the Highest-grossing films table.
    
    Args:
        html_content (str): HTML content of the Wikipedia page
        
    Returns:
        list: List of dictionaries containing film data
    """
    # Parse HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the first table with class 'wikitable'
    table = soup.find('table', class_='wikitable')
    
    if not table:
        return []
    
    # Initialize list to store results
    films = []
    
    # Get headers
    headers = []
    header_row = table.find('tr')
    if header_row:
        for th in header_row.find_all('th'):
            # Clean header text
            header = th.text.strip().lower()
            # Convert to more readable column names
            header = header.replace('\n', ' ').replace('♦', '').strip()
            headers.append(header)
    
    # Process each row
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all(['td', 'th'])
        
        if len(cols) >= len(headers):
            film_data = {}
            
            for i, col in enumerate(cols):
                # Extract text and clean it
                value = preprocess_text(col.text.strip())
                # Add to film data dictionary
                if i < len(headers):
                    film_data[headers[i]] = value
                
                wiki_page_link = row.find('i').find('a')
                if wiki_page_link:
                    film_data['wiki_page_link'] = wiki_page_link.get('href')
            
            films.append(film_data)
    
    return films

In [4]:
def extract_film_details(html_content):
    """
    Extract Box office, Country and Directed by information from a film's Wikipedia page.
    
    Args:
        html_content (str): HTML content of the Wikipedia page
        
    Returns:
        dict: Dictionary containing box_office, country, and director information
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Initialize result dictionary
    result = {
        'box_office': None,
        'country': None,
        'director': None
    }
    
    # Find the infobox
    infobox = soup.find('table', class_='infobox')
    if infobox:
        # Extract data from infobox
        rows = infobox.find_all('tr')
        for row in rows:
            header = row.find('th')
            if header:
                header_text = header.get_text().strip().lower()
                
                # Extract country
                if 'country' in header_text or 'countries' in header_text:
                    value = row.find('td')
                    if value:
                        country_list = value.find('ul')
                        if country_list:
                            countries = [preprocess_text(li.get_text().strip()) for li in country_list.find_all('li')]
                        else:
                            countries = [preprocess_text(value.get_text().strip())]
                        result['country'] = countries
                
                # Extract director
                elif 'directed by' in header_text:
                    value = row.find('td')
                    if value:
                        # Check if there's a list of directors
                        director_list = value.find('ul')
                        if director_list:
                            # Extract directors from list items
                            directors = [preprocess_text(li.get_text().strip()) for li in director_list.find_all('li')]
                        else:
                            # Single director
                            directors = [preprocess_text(value.get_text().strip())]
                        result['director'] = directors
    
    return result

### Database design

In this assignment, I've used SQLModel ORM to create and interact with database

In [5]:
class Film(SQLModel, table=True):
    """
    A film from the highest-grossing films list with additional details
    """
    __tablename__ = "films"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    title: str = Field(nullable=False)
    release_year: Optional[int] = Field(default=None)
    director: Optional[str] = Field(default=None)
    box_office: Optional[str] = Field(default=None)
    country_of_origin: Optional[str] = Field(default=None) 


### Extraction and preprocessing

In [6]:
# Get the highest-grossing films list
films_list_page_response = requests.get("https://en.wikipedia.org/wiki/List_of_highest-grossing_films")
films_list_page_response.raise_for_status()
films_list_page = films_list_page_response.text

In [7]:
# Extract the films from the highest-grossing films list
films = extract_highest_grossing_films(films_list_page)

In [8]:
# Extract the additional details from the films and create a list of Film objects
processed_films = []

for film in films:
    film_page_response = requests.get(f"https://en.wikipedia.org{film['wiki_page_link']}")
    film_page_response.raise_for_status()
    film_page = film_page_response.text
    film_details = extract_film_details(film_page)
    processed_films.append(Film(
        title=film['title'],
        release_year=film['year'],
        box_office=film['worldwide gross'],
        country_of_origin=", ".join(film_details['country']),
        director=", ".join(film_details['director'])
    ))
    

### Save processed films to JSON for Frontend

In [9]:
film_dumps = [film.model_dump(exclude={"id"}) for film in processed_films]
with open("films.json", "w") as f:
    json.dump(film_dumps, f)

/Users/nikita/anaconda3/envs/rag_env/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` with value `'2009'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/nikita/anaconda3/envs/rag_env/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` with value `'2019'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/nikita/anaconda3/envs/rag_env/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` with value `'2022'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/nikita/anaconda3/envs/rag_env/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` with value `'1997'`

### SQLite database creation and interaction
For this assignment, I've used SQLite as simple in-memory database that doesn't require any additional deployment.
Such decision accelerate prototyping and simplifies deployment.

In [10]:
# SQLite URL
SQLITE_URL = "sqlite:///./database.db"

# Create engine
engine = create_engine(
    SQLITE_URL,
    echo=True,  # Set to False in production
    connect_args={"check_same_thread": False}  # Needed for SQLite
)
SQLModel.metadata.drop_all(engine)
SQLModel.metadata.create_all(engine)

2025-02-19 13:35:22,331 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-19 13:35:22,332 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("films")
2025-02-19 13:35:22,332 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-19 13:35:22,333 INFO sqlalchemy.engine.Engine 
DROP TABLE films
2025-02-19 13:35:22,333 INFO sqlalchemy.engine.Engine [no key 0.00028s] ()
2025-02-19 13:35:22,334 INFO sqlalchemy.engine.Engine COMMIT
2025-02-19 13:35:22,334 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-19 13:35:22,335 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("films")
2025-02-19 13:35:22,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-19 13:35:22,336 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("films")
2025-02-19 13:35:22,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-19 13:35:22,337 INFO sqlalchemy.engine.Engine 
CREATE TABLE films (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	release_year INTEGER, 
	director VARCHAR, 
	box_office VARCHAR, 


In [11]:
with Session(engine) as session:
    session.add_all(processed_films)
    session.commit()

2025-02-19 13:35:22,342 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-19 13:35:22,344 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country_of_origin) VALUES (?, ?, ?, ?, ?) RETURNING id
2025-02-19 13:35:22,344 INFO sqlalchemy.engine.Engine [generated in 0.00012s (insertmanyvalues) 1/50 (ordered; batch not supported)] ('Avatar', '2009', 'James Cameron', '$2,923,706,026', 'United Kingdom, United States')
2025-02-19 13:35:22,345 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country_of_origin) VALUES (?, ?, ?, ?, ?) RETURNING id
2025-02-19 13:35:22,346 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/50 (ordered; batch not supported)] ('Avengers: Endgame', '2019', 'Anthony Russo, Joe Russo', '$2,797,501,328', 'United States')
2025-02-19 13:35:22,347 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country_of_origin) VALUES (?, ?, ?, ?, ?) RETURNI